In [ ]:
import os
import sys
import traci
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")

SUMO_CMD = ["sumo", "-c", "203.sumocfg", "--no-step-log", "true"]
TLS_ID = "622024959"
LANES_NS = ["-1226913076_0", "-1226913076_1", "-E10_0", "-E10_1"]
LANES_WE = ["E0_0", "E0_1", "-E1_0", "-E1_1"]
ALL_LANES = LANES_NS + LANES_WE

MIN_GREEN = 15  
THRESHOLD = 6   
CHECK_FREQ = 5  

def get_current_stats():
    q_len = sum(traci.lane.getLastStepHaltingNumber(lane) for lane in ALL_LANES)
    w_time = sum(traci.lane.getWaitingTime(lane) for lane in ALL_LANES)
    speeds = [traci.vehicle.getSpeed(v) for lane in ALL_LANES for v in traci.lane.getLastStepVehicleIDs(lane)]
    avg_speed = np.mean(speeds) if speeds else 0.0
    return q_len, w_time, avg_speed

def run_simulation(mode):
    traci.start(SUMO_CMD)
    step = 0
    last_change_step = 0
    data = []

    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        
        if mode == "AI" and step % CHECK_FREQ == 0:
            phase = traci.trafficlight.getPhase(TLS_ID)
            ns_halthing = sum(traci.lane.getLastStepHaltingNumber(l) for l in LANES_NS)
            we_halthing = sum(traci.lane.getLastStepHaltingNumber(l) for l in LANES_WE)
            
            time_in_phase = step - last_change_step

            if phase == 0:
                if time_in_phase > MIN_GREEN and we_halthing > ns_halthing + 2 and we_halthing > THRESHOLD:
                    traci.trafficlight.setPhase(TLS_ID, 1)
                    last_change_step = step
                elif ns_halthing > THRESHOLD:
                    traci.trafficlight.setPhaseDuration(TLS_ID, 10)
            
            elif phase == 2: 
                if time_in_phase > MIN_GREEN and ns_halthing > we_halthing + 2 and ns_halthing > THRESHOLD:
                    traci.trafficlight.setPhase(TLS_ID, 3)
                    last_change_step = step
                elif we_halthing > THRESHOLD:
                    traci.trafficlight.setPhaseDuration(TLS_ID, 10)

        q, w, s = get_current_stats()
        data.append([step, q, w, s])
        step += 1

    traci.close()
    return pd.DataFrame(data, columns=["Step", "Queue", "WaitTime", "Speed"])

print("Запуск: Static...")
df_static = run_simulation("STATIC")
print("Запуск: Optimized AI...")
df_ai = run_simulation("AI")
df_ai.to_excel("test.xlsx")

fig, axs = plt.subplots(3, 1, figsize=(10, 12))
colors = {'STATIC': 'red', 'AI': 'green'}

for i, (col, title, ylabel) in enumerate([
    ("Queue", "Длина очереди (меньше = лучше)", "Количество авто"),
    ("WaitTime", "Общее время ожидания", "Секунды"),
    ("Speed", "Средняя скорость (выше = лучше)", "м/с")
]):
    axs[i].plot(df_static["Step"], df_static[col].rolling(20).mean(), label="Статичный", color=colors['STATIC'])
    axs[i].plot(df_ai["Step"], df_ai[col].rolling(20).mean(), label="Оптимизированный ИИ", color=colors['AI'])
    axs[i].set_title(title)
    axs[i].set_ylabel(ylabel)
    axs[i].legend()
    axs[i].grid(True)

plt.tight_layout()
plt.show()

Запуск: Static...
 Retrying in 1 seconds


Error: Could not access configuration '203.sumocfg'.
Quitting (on error).
/home/lorriant/Рабочий стол/rmp10a/myvenv/lib/python3.12/site-packages/traci/main.py:156: UserWarning: Could not connect to TraCI server using port 40409 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +
Error: Could not access configuration '203.sumocfg'.
Quitting (on error).


 Retrying in 1 seconds


/home/lorriant/Рабочий стол/rmp10a/myvenv/lib/python3.12/site-packages/traci/main.py:156: UserWarning: Could not connect to TraCI server using port 42891 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +
Error: Could not access configuration '203.sumocfg'.
Quitting (on error).


 Retrying in 1 seconds


...

 Retrying in 1 seconds


/home/lorriant/Рабочий стол/rmp10a/myvenv/lib/python3.12/site-packages/traci/main.py:156: UserWarning: Could not connect to TraCI server using port 40593 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +
Error: Could not access configuration '203.sumocfg'.
Quitting (on error).


 Retrying in 1 seconds


/home/lorriant/Рабочий стол/rmp10a/myvenv/lib/python3.12/site-packages/traci/main.py:156: UserWarning: Could not connect to TraCI server using port 44935 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +
Error: Could not access configuration '203.sumocfg'.
Quitting (on error).


 Retrying in 1 seconds
